# Deep learning framework example: Clickbate dataset

**Important**: Please see the [MNIST notebook example](https://github.com/plandes/deeplearn/blob/master/notebook/mnist.ipynb) in the [zensols.deeplearn](https://github.com/plandes/deeplearn) API first, as it contains more explaination of how the framework is used.

See the [saved notebook](https://htmlpreview.github.io/?https://github.com/plandes/deepnlp/blob/master/example/clickbate/notebook/clickbate.html) to see the output of this example.

In [ ]:
# environemnt configuration and set up: add this (deepnlp) library to the Python path and framework entry point
from harness import NotebookHarness
harness = NotebookHarness()
mng = harness()

## Print information about 

Use the factory to create the model executor.  The `write` method gives statistics on the data set that is configured on the executor.

In [ ]:
from io import StringIO
# read the configuration from glove.conf using the same command line process to load the config and models
facade = mng.create_facade('glove50')
from zensols.config import Writable
# set indention level for human readable (pretty print like) output
Writable.WRITABLE_INDENT_SPACE = 2
facade.write()

## Tune hyperparameters

Set model parameters to get a feel for where they need to be before changing features.

In [ ]:
facade.epochs = 20
facade.language_attributes = set()
mng.run()
facade.persist_result()

## Add linguistic features

Add spaCy generated lingustic features by appending them to the embedding layer.  The enumerations are one-hot encoded vectors of POS tags, NER entities and dependenccy parent/child relationship.

In [ ]:
facade.language_attributes = {'enums'}
mng.run()
facade.persist_result()

## More linguistic features

Add the syntactic dependency parser parent/child relationship as a feature.

In [ ]:
facade.language_attributes = {'enums', 'dependencies'}
mng.run()
facade.persist_result()

## Transformer

Now we'll try contextual BERT embeddings.  When we use `create_facade` it resets the language features and embedings to what's configured in `transformer.conf`.  In this case, it's a Bert cased model using the pooler output for the classification task.

In [ ]:
facade = mng.create_facade('transformer')
mng.run()
facade.persist_result()

# RoBERTa

Try the RoBERTa model for comparison; this is done by overriding the ``model_id`` option on the ``transformer_resource`` section.

In [ ]:
mng.config('transformer_resource', model_id='roberta-base')
facade = mng.create_facade('transformer')
facade.epochs = 10
mng.run()

# DistilBERT

Try the DistilBERT model for comparison in the same way as RoBERTa.

In [ ]:
mng.config('transformer_resource', model_id='distilbert-base-cased')
facade = mng.create_facade('transformer')
mng.run()

# Compare results

Generate a dataframe with the performance metrics of the previous runs.

In [ ]:
from zensols.deeplearn.result import ModelResultManager, ModelResultReporter
rm: ModelResultManager = facade.result_manager
reporter = ModelResultReporter(rm)
reporter.dataframe